In [ ]:
import numpy as np
import k3d
import open3d as o3d

In [ ]:
points = np.load('point_clouds/office_0_points.npy')
print("Point Cloud shape:", points.shape)

#sparse_pc = o3d.io.read_point_cloud('data/flight/sparse_pc.ply')
#points2 = np.asarray(sparse_pc.points)
#print(points2.shape) # the point cloud they have supplied

# normalizing
# centroid = points.mean(axis=0)
# centered_points = points - centroid
# max_abs = np.max(np.abs(centered_points), axis=0)
# max_abs[max_abs == 0] = 1
# points = centered_points / max_abs

min_x, min_y, min_z = points.min(axis=0)
max_x, max_y, max_z = points.max(axis=0)

print(f"X Range: {min_x} to {max_x}")
print(f"Y Range: {min_y} to {max_y}")
print(f"Z Range: {min_z} to {max_z}")

points = np.asarray(points) # convert back to numpy array



In [ ]:
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points)
# print("Count:", len(pcd.points))

# # Bounding box filtering
# min_bound = np.array([-np.inf, -0.6, -0.8])
# max_bound = np.array([np.inf, 0.6, 1])
# # min_bound = np.array([-np.inf, -0.6, -0.8])
# # max_bound = np.array([np.inf, 0.6, 0.2])
# bounding_box = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
# pcd = pcd.crop(bounding_box)
# print("Count:", len(pcd.points))

# # Downsampling
# # pcd = pcd.voxel_down_sample(0.01)  # Adjust size based on detail needs
# print("Count:", len(pcd.points))


# # Remove Random Noise
# # pcd, inliers = pcd.remove_statistical_outlier(nb_neighbors=50, std_ratio=2.0)
# print("Count:", len(pcd.points))


# # Remove Isolated Points
# # pcd, inliers = pcd.remove_radius_outlier(nb_points=8, radius=0.5)
# print("Count:", len(pcd.points))


# points = np.asarray(pcd.points) # convert back to numpy array
# print("Point Cloud shape:", points.shape) # point cloud after cleaning up


In [ ]:
# points = np.asarray(pcd.points)
# centroid = points.mean(axis=0)
# centered_points = points - centroid
# min_vals = points.min(axis=0)
# max_vals = points.max(axis=0)
# ranges = max_vals - min_vals
# ranges[ranges == 0] = 1
# normalized_points = centered_points / (ranges / 2)
# points = normalized_points


In [ ]:
# downsampling 
# def downsample_point_cloud_random(point_cloud, num_points):
#     if point_cloud.shape[0] <= num_points:
#         return point_cloud

#     indices = np.random.choice(point_cloud.shape[0], num_points, replace=False)
#     return point_cloud[indices]

# points = downsample_point_cloud_random(points, 100000) * 100
# print("Point Cloud shape", points.shape) # point cloud after downsampling

In [ ]:
print("Point Cloud shape", points.shape) # point cloud before plotting
plot = k3d.plot()

plt_points = k3d.points(positions=points.astype(np.float32),
                        point_size=0.1)

plot += plt_points

mean_config = np.mean(points, axis=0)
center_sphere = k3d.points(positions=np.array([mean_config], dtype=np.float32), color=0xff0000, point_size=0.3)
plot += center_sphere

plot.display()

In [1]:
import json, numpy as np, open3d as o3d, k3d

# === Helpers
def load_ply_as_numpy(path):
    pcd = o3d.io.read_point_cloud(path)
    return np.asarray(pcd.points)

# === File Paths
gsplat_path    = "point_clouds/office_3_points.npy"
traj_json_path = "trajs/office_3_ball-to-ellipsoid.json"

# === Load Data
gsplat_pts = np.load(gsplat_path).astype(np.float32)

with open(traj_json_path) as f:
    traj_data = json.load(f)

# Extract trajectories, success flags, and safety arrays
trajectories = []
success_flags = []
safety_counts = []

for i, block in enumerate(traj_data["total_data"]):
    traj = np.asarray(block["traj"], dtype=np.float32)[:, :3]   # (N,3)
    trajectories.append(traj)

    success = block["sucess"][0]  # typo "sucess" in your JSON
    success_flags.append(success)

    # Safety analysis
    safety_vals = np.array(block["safety"], dtype=np.float32)
    num_unsafe = np.sum(safety_vals < 0)
    safety_counts.append(num_unsafe)

    if num_unsafe > 0:
        print(f"Trajectory {i}: ❗ {num_unsafe} unsafe steps | {'✅ Success' if success else '❌ Fail'}")
    else:
        print(f"Trajectory {i}: ✅ All safe | {'✅ Success' if success else '❌ Fail'}")

print("\nSummary:")
print(f"Total trajectories: {len(trajectories)}")
print(f"Trajectories with unsafe moments: {sum(s > 0 for s in safety_counts)} / {len(trajectories)}")

# === Create Plot
plot = k3d.plot(grid_visible=False, axes_helper=1)

# Plot point cloud
plot += k3d.points(gsplat_pts, point_size=0.01, color=0x00ff00, shader='3d')  # Green

# Plot trajectories
for traj, success, unsafe_count in zip(trajectories, success_flags, safety_counts):
    if unsafe_count > 0:
        color = 0xffa500  # Orange = had unsafe events
    else:
        color = 0x0000ff #if success else 0xff0000  # Blue = success, Red = fail
    plot += k3d.line(traj, width=0.003, color=color)

plot.display()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Trajectory 0: ✅ All safe | ❌ Fail
Trajectory 1: ✅ All safe | ❌ Fail
Trajectory 2: ✅ All safe | ❌ Fail
Trajectory 3: ✅ All safe | ✅ Success
Trajectory 4: ✅ All safe | ❌ Fail
Trajectory 5: ✅ All safe | ❌ Fail
Trajectory 6: ✅ All safe | ❌ Fail
Trajectory 7: ✅ All safe | ❌ Fail
Trajectory 8: ✅ All safe | ❌ Fail
Trajectory 9: ✅ All safe | ❌ Fail
Trajectory 10: ✅ All safe | ❌ Fail
Trajectory 11: ✅ All safe | ❌ Fail
Trajectory 12: ❗ 8 unsafe steps | ❌ Fail
Trajectory 13: ✅ All safe | ❌ Fail
Trajectory 14: ✅ All safe | ✅ Success
Trajectory 15: ✅ All safe | ❌ Fail
Trajectory 16: ✅ All safe | ✅ Success
Trajectory 17: ✅ All safe | ❌ Fail
Trajectory 18: ✅ All safe | ❌ Fail
Trajectory 19: ✅ All safe | ❌ Fail
Trajectory 20: ✅ All safe | ❌ Fail
Trajectory 21: ✅ All safe | ❌ Fail
Trajectory 22: ✅ All safe | ✅ Success
Trajectory 23: ✅ A

Output()

In [ ]:
# === View ONLY Failed Trajectories ===

import json
import numpy as np

# === Load Data
traj_json_path = "trajs/office_3_ball-to-ellipsoid.json"

with open(traj_json_path) as f:
    traj_data = json.load(f)

# === Prepare Lists
total_trajectories = 0
failed_trajectories = []

for idx, block in enumerate(traj_data["total_data"]):
    success = block["sucess"][0]   # typo "sucess" in your file
    safety_vals = np.array(block["safety"], dtype=np.float32)
    num_unsafe = np.sum(safety_vals < 0)

    if not success:
        # It's a failed trajectory
        status_line = (f"Trajectory {idx}: "
                       f"{'❗ ' + str(num_unsafe) + ' unsafe steps' if num_unsafe > 0 else '✅ All safe'} | "
                       f"❌ Fail")
        failed_trajectories.append(status_line)

    total_trajectories += 1

# === Print Results
print("\n".join(failed_trajectories))

print("\nSummary:")
print(f"Total trajectories        : {total_trajectories}")
print(f"Failed trajectories printed: {len(failed_trajectories)}")


Trajectory 0: ✅ All safe | ❌ Fail
Trajectory 1: ✅ All safe | ❌ Fail
Trajectory 2: ❗ 17 unsafe steps | ❌ Fail
Trajectory 3: ❗ 24 unsafe steps | ❌ Fail
Trajectory 4: ✅ All safe | ❌ Fail
Trajectory 5: ✅ All safe | ❌ Fail
Trajectory 6: ✅ All safe | ❌ Fail
Trajectory 7: ✅ All safe | ❌ Fail
Trajectory 8: ❗ 26 unsafe steps | ❌ Fail
Trajectory 9: ✅ All safe | ❌ Fail
Trajectory 10: ❗ 22 unsafe steps | ❌ Fail
Trajectory 12: ✅ All safe | ❌ Fail
Trajectory 13: ✅ All safe | ❌ Fail
Trajectory 14: ❗ 18 unsafe steps | ❌ Fail
Trajectory 15: ❗ 35 unsafe steps | ❌ Fail
Trajectory 17: ❗ 18 unsafe steps | ❌ Fail
Trajectory 18: ❗ 21 unsafe steps | ❌ Fail
Trajectory 19: ❗ 24 unsafe steps | ❌ Fail
Trajectory 20: ✅ All safe | ❌ Fail
Trajectory 21: ❗ 11 unsafe steps | ❌ Fail
Trajectory 22: ❗ 21 unsafe steps | ❌ Fail
Trajectory 23: ❗ 21 unsafe steps | ❌ Fail
Trajectory 24: ❗ 23 unsafe steps | ❌ Fail
Trajectory 25: ✅ All safe | ❌ Fail
Trajectory 26: ✅ All safe | ❌ Fail
Trajectory 27: ❗ 37 unsafe steps | ❌ Fail
Tr

In [ ]:
### VIEW SCENE ALIGNMENT

import json
import numpy as np
import open3d as o3d
import k3d
from coords import replica2gsplat

def load_ply_as_numpy(path):
    pcd = o3d.io.read_point_cloud(path)
    return np.asarray(pcd.points)

replica_path = "/scratch/auj7tx/vmap/office_4/mesh.ply"
gsplat_path = "point_clouds/office_4_points.npy"
# traj_json_path = "/home/auj7tx/cral/safer-splat/trajs/room_0_ball-to-ellipsoid.json"
scene_name = "office_4"

replica_pts = load_ply_as_numpy(replica_path).astype(np.float32)
gsplat_pts = np.load(gsplat_path).astype(np.float32)

# Transform Replica points using the imported function via list comprehension
replica_pts_transformed = np.array([replica2gsplat(p, scene_name) for p in replica_pts]).astype(np.float32)

# --- Define Start/End points in ORIGINAL Replica coordinates ---
start_point_replica = np.array([-1.270207247, -1.988134284, -1.0], dtype=np.float32)
end_point_replica   = np.array([-0.283352436,  0.56045748,  -1.0], dtype=np.float32)

# --- Transform the Start/End points ---
start_point_gsplat = replica2gsplat(start_point_replica, scene_name)
end_point_gsplat   = replica2gsplat(end_point_replica, scene_name)


# Trajectory loading commented out
# with open(traj_json_path) as f:
#     traj_data = json.load(f)
# trajectories = [
#     np.asarray(block["traj"], dtype=np.float32)[:, :3]
#     for block in traj_data["total_data"]
# ]

print(f"Replica (Transformed): {replica_pts_transformed.shape[0]:,} points")
print(f"GSplat               : {gsplat_pts.shape[0]:,} points")

plot = k3d.plot(grid_visible=False, axes_helper=1.0)

# Plot the main point clouds
plot += k3d.points(replica_pts_transformed, point_size=0.01, color=0xff0000, shader='3d', name='Transformed Replica') # Red
plot += k3d.points(gsplat_pts, point_size=0.01, color=0x00ff00, shader='3d', name='GSplat') # Green

# --- Plot the transformed Start/End points ---
if start_point_gsplat is not None:
    # k3d expects Nx3 array, so reshape the single point to (1, 3)
    plot += k3d.points(start_point_gsplat.reshape(1, 3), point_size=0.1, color=0x0000ff, shader='3d', name='Start Point') # Blue
    print(f"\nStart point (GSplat):    ", start_point_gsplat)
else:
    print("\nCould not transform start point.")

if end_point_gsplat is not None:
    plot += k3d.points(end_point_gsplat.reshape(1, 3), point_size=0.1, color=0xffa500, shader='3d', name='End Point') # Orange
    print(f"End point (GSplat):      ", end_point_gsplat)
else:
    print("Could not transform end point.")


# Optional trajectory plotting commented out
# if 'trajectories' in locals():
#    for i, traj in enumerate(trajectories):
#        plot += k3d.line(traj, width=0.01, color=0x0000ff, name=f'Trajectory {i}')

print(f"\nTransformed Replica center:", np.mean(replica_pts_transformed, axis=0))
print(f"GSplat center:             ", np.mean(gsplat_pts, axis=0))

plot.display()

Replica: 1,187,140 points
GSplat:  222,484 points
Loaded 50 trajectories


Output()

In [ ]:
## ADJUST SCENE ALIGNMENT

import k3d
import numpy as np
import open3d as o3d
import ipywidgets as widgets
from IPython.display import display
from scipy.spatial.transform import Rotation as R

# === Load Point Clouds ===
def load_ply_as_numpy(path):
    pcd = o3d.io.read_point_cloud(path)
    return np.asarray(pcd.points)

replica_path = "/scratch/auj7tx/vmap/office_4/mesh.ply"
gsplat_path = "point_clouds/office_4_points.npy"

replica_pts_raw = load_ply_as_numpy(replica_path).astype(np.float32)
gsplat_pts = np.load(gsplat_path).astype(np.float32)

# === Downsample for visualization ===
def downsample_point_cloud_random(point_cloud, num_points):
    if point_cloud.shape[0] <= num_points:
        return point_cloud
    indices = np.random.choice(point_cloud.shape[0], num_points, replace=False)
    return point_cloud[indices]

#replica_pts = downsample_point_cloud_random(replica_pts_raw, 100000)
gsplat_pts = downsample_point_cloud_random(gsplat_pts, 100000)

# === Setup K3D Plot ===
plot = k3d.plot(grid_visible=True, axes_helper=1)

replica_obj = k3d.points(replica_pts, point_size=0.01, color=0xff0000, shader='3d')
gsplat_obj = k3d.points(gsplat_pts, point_size=0.01, color=0x00ff00, shader='3d')

plot += gsplat_obj
plot += replica_obj

# === Sliders ===
def make_slider(min_, max_, step, value, description):
    return widgets.FloatSlider(min=min_, max=max_, step=step, value=value, description=description, continuous_update=True)

slider_tx = make_slider(-5, 5, 0.01, 0.0, 'TX')
slider_ty = make_slider(-5, 5, 0.01, 0.0, 'TY')
slider_tz = make_slider(-5, 5, 0.01, 0.0, 'TZ')

slider_rx = make_slider(-180, 180, 1, 0.0, 'RotX')
slider_ry = make_slider(-180, 180, 1, 0.0, 'RotY')
slider_rz = make_slider(-180, 180, 1, 0.0, 'RotZ')

slider_scale = make_slider(0.01, 2.0, 0.01, 1.0, 'Scale')

# === Update function ===
def update_replica(change=None):
    tx, ty, tz = slider_tx.value, slider_ty.value, slider_tz.value
    rx, ry, rz = np.deg2rad(slider_rx.value), np.deg2rad(slider_ry.value), np.deg2rad(slider_rz.value)
    scale = slider_scale.value

    rotation = R.from_euler('xyz', [rx, ry, rz]).as_matrix()
    transformed = (rotation @ (replica_pts.T * scale)).T + np.array([tx, ty, tz])
    replica_obj.positions = transformed.astype(np.float32)

# Attach handlers
for s in [slider_tx, slider_ty, slider_tz, slider_rx, slider_ry, slider_rz, slider_scale]:
    s.observe(update_replica, names='value')

# === Button Helpers ===
def make_inc_dec(slider, delta):
    def inc(_):
        slider.value += delta
    def dec(_):
        slider.value -= delta
    return inc, dec

def slider_with_buttons(slider, delta):
    inc, dec = make_inc_dec(slider, delta)
    inc_button = widgets.Button(description="+", layout=widgets.Layout(width='30px'))
    dec_button = widgets.Button(description="-", layout=widgets.Layout(width='30px'))
    inc_button.on_click(inc)
    dec_button.on_click(dec)
    return widgets.HBox([slider, dec_button, inc_button])

# === Organize all controls ===
controls = widgets.VBox([
    slider_with_buttons(slider_tx, 0.01),
    slider_with_buttons(slider_ty, 0.01),
    slider_with_buttons(slider_tz, 0.01),
    slider_with_buttons(slider_rx, 1),
    slider_with_buttons(slider_ry, 1),
    slider_with_buttons(slider_rz, 1),
    slider_with_buttons(slider_scale, 0.01),
])

display(controls)
plot.display()

